In [1]:
# Imports

import os
import warnings
import numpy as np

import tensorflow as tf

from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Bidirectional

from data_repository import DataRepository
from sklearn.model_selection import KFold, StratifiedKFold

# Ignore future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Root CSV files directory
dirname = "./data/absolute/2D/"  

repo = DataRepository(dirname, verbose=False)

#load all
x, y = repo.getDataAndLabels()

['Computer', 'Deutschland', 'du', 'Haben', 'Hallo', 'ich', 'Mainz', 'Software', 'unser', 'Welt', 'zeigen']


In [3]:
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Num GPUs: 1


In [4]:
with open('tokens_json.txt', 'r') as outfile:
    json_ex = outfile.read()
    

tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(json_ex)
print(tokenizer.word_index)
token_labels = {y:x for x,y in tokenizer.word_index.items()}

y_integer = np.argmax(y, axis=1)
y_name= ([token_labels[p] for p in y_integer])


{'computer': 1, 'deutschland': 2, 'du': 3, 'haben': 4, 'hallo': 5, 'ich': 6, 'mainz': 7, 'software': 8, 'unser': 9, 'welt': 10, 'zeigen': 11}


In [5]:
y_name

['computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',
 'computer',

In [15]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
cvscores = []
for train, test in skfold.split(x, y_name):
    print("size_x_train: ", len(x[train]))
    print("size_y_train: ", len(y[train]))
    print("size_x_test: ", len(x[test]))
    print("size_y_test: ", len(y[test]))
    model = Sequential()
    model.add(layers.LSTM(128, return_sequences=True, input_shape=(x.shape[1], x.shape[2])))
    model.add(layers.LSTM(64, return_sequences=True)) 
    model.add(layers.LSTM(96))  
    model.add(layers.Dense(12, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
    model.summary()

    history=model.fit(x[train],y[train],epochs=170,validation_data=(x[test],y[test]),verbose=2)
    scores = model.evaluate(x[test], y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    print("Scores: ", scores)
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))





size_x_train:  743
size_y_train:  743
size_x_test:  186
size_y_test:  186
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 100, 128)          110080    
_________________________________________________________________
lstm_13 (LSTM)               (None, 100, 64)           49408     
_________________________________________________________________
lstm_14 (LSTM)               (None, 96)                61824     
_________________________________________________________________
dense_4 (Dense)              (None, 12)                1164      
Total params: 222,476
Trainable params: 222,476
Non-trainable params: 0
_________________________________________________________________
Epoch 1/170
24/24 - 1s - loss: 2.3553 - accuracy: 0.1548 - precision_4: 0.6316 - recall_4: 0.0162 - val_loss: 2.0460 - val_accuracy: 0.2258 - val_precision_4: 1.0000 - val_

Epoch 40/170
24/24 - 0s - loss: 1.3708 - accuracy: 0.4670 - precision_4: 0.7063 - recall_4: 0.2719 - val_loss: 1.3476 - val_accuracy: 0.4839 - val_precision_4: 0.6984 - val_recall_4: 0.2366
Epoch 41/170
24/24 - 0s - loss: 1.3750 - accuracy: 0.4751 - precision_4: 0.6779 - recall_4: 0.2719 - val_loss: 1.2850 - val_accuracy: 0.4785 - val_precision_4: 0.7581 - val_recall_4: 0.2527
Epoch 42/170
24/24 - 0s - loss: 1.3155 - accuracy: 0.4939 - precision_4: 0.7407 - recall_4: 0.2423 - val_loss: 1.4288 - val_accuracy: 0.4731 - val_precision_4: 0.6515 - val_recall_4: 0.2312
Epoch 43/170
24/24 - 0s - loss: 1.3057 - accuracy: 0.4939 - precision_4: 0.7163 - recall_4: 0.2786 - val_loss: 1.2096 - val_accuracy: 0.5430 - val_precision_4: 0.8095 - val_recall_4: 0.2742
Epoch 44/170
24/24 - 0s - loss: 1.2490 - accuracy: 0.5034 - precision_4: 0.7517 - recall_4: 0.2934 - val_loss: 1.3174 - val_accuracy: 0.4892 - val_precision_4: 0.6892 - val_recall_4: 0.2742
Epoch 45/170
24/24 - 0s - loss: 1.2847 - accuracy:

Epoch 84/170
24/24 - 0s - loss: 0.8305 - accuracy: 0.6958 - precision_4: 0.7653 - recall_4: 0.6231 - val_loss: 1.1426 - val_accuracy: 0.6129 - val_precision_4: 0.7368 - val_recall_4: 0.5269
Epoch 85/170
24/24 - 0s - loss: 0.8829 - accuracy: 0.6581 - precision_4: 0.8037 - recall_4: 0.5289 - val_loss: 0.8660 - val_accuracy: 0.6989 - val_precision_4: 0.7820 - val_recall_4: 0.5591
Epoch 86/170
24/24 - 0s - loss: 0.7858 - accuracy: 0.6918 - precision_4: 0.8029 - recall_4: 0.5868 - val_loss: 0.7521 - val_accuracy: 0.6935 - val_precision_4: 0.8102 - val_recall_4: 0.5968
Epoch 87/170
24/24 - 0s - loss: 0.7723 - accuracy: 0.7052 - precision_4: 0.8239 - recall_4: 0.5855 - val_loss: 0.7657 - val_accuracy: 0.6935 - val_precision_4: 0.7714 - val_recall_4: 0.5806
Epoch 88/170
24/24 - 0s - loss: 0.6879 - accuracy: 0.7335 - precision_4: 0.8280 - recall_4: 0.6285 - val_loss: 0.6882 - val_accuracy: 0.7527 - val_precision_4: 0.8456 - val_recall_4: 0.6774
Epoch 89/170
24/24 - 0s - loss: 0.6094 - accuracy:

Epoch 127/170
24/24 - 0s - loss: 0.4622 - accuracy: 0.8345 - precision_4: 0.8949 - recall_4: 0.7564 - val_loss: 0.7514 - val_accuracy: 0.7796 - val_precision_4: 0.8148 - val_recall_4: 0.7097
Epoch 128/170
24/24 - 0s - loss: 0.4729 - accuracy: 0.8210 - precision_4: 0.8801 - recall_4: 0.7604 - val_loss: 1.1189 - val_accuracy: 0.6613 - val_precision_4: 0.7169 - val_recall_4: 0.6398
Epoch 129/170
24/24 - 0s - loss: 0.5766 - accuracy: 0.7968 - precision_4: 0.8270 - recall_4: 0.7591 - val_loss: 0.7039 - val_accuracy: 0.7419 - val_precision_4: 0.8012 - val_recall_4: 0.7151
Epoch 130/170
24/24 - 0s - loss: 0.4904 - accuracy: 0.8008 - precision_4: 0.8609 - recall_4: 0.7497 - val_loss: 0.6939 - val_accuracy: 0.7796 - val_precision_4: 0.8562 - val_recall_4: 0.7366
Epoch 131/170
24/24 - 0s - loss: 0.5572 - accuracy: 0.7873 - precision_4: 0.8483 - recall_4: 0.7227 - val_loss: 0.7382 - val_accuracy: 0.7527 - val_precision_4: 0.7849 - val_recall_4: 0.7258
Epoch 132/170
24/24 - 0s - loss: 0.5441 - acc

Epoch 170/170
24/24 - 0s - loss: 0.2075 - accuracy: 0.9139 - precision_4: 0.9264 - recall_4: 0.8977 - val_loss: 0.6389 - val_accuracy: 0.8118 - val_precision_4: 0.8197 - val_recall_4: 0.8065
accuracy: 81.18%
size_x_train:  743
size_y_train:  743
size_x_test:  186
size_y_test:  186
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 100, 128)          110080    
_________________________________________________________________
lstm_16 (LSTM)               (None, 100, 64)           49408     
_________________________________________________________________
lstm_17 (LSTM)               (None, 96)                61824     
_________________________________________________________________
dense_5 (Dense)              (None, 12)                1164      
Total params: 222,476
Trainable params: 222,476
Non-trainable params: 0
______________________________

...

Epoch 32/170
24/24 - 0s - loss: 1.4899 - accuracy: 0.4194 - precision_8: 0.6700 - recall_8: 0.1801 - val_loss: 1.6507 - val_accuracy: 0.3730 - val_precision_8: 0.5556 - val_recall_8: 0.1351
Epoch 33/170
24/24 - 0s - loss: 1.4792 - accuracy: 0.4409 - precision_8: 0.6537 - recall_8: 0.2030 - val_loss: 1.4456 - val_accuracy: 0.4162 - val_precision_8: 0.6618 - val_recall_8: 0.2432
Epoch 34/170
24/24 - 0s - loss: 1.3990 - accuracy: 0.4718 - precision_8: 0.6910 - recall_8: 0.2675 - val_loss: 1.4123 - val_accuracy: 0.4378 - val_precision_8: 0.6118 - val_recall_8: 0.2811
Epoch 35/170
24/24 - 0s - loss: 1.3660 - accuracy: 0.4879 - precision_8: 0.6930 - recall_8: 0.2944 - val_loss: 1.5048 - val_accuracy: 0.4162 - val_precision_8: 0.5463 - val_recall_8: 0.3189
Epoch 36/170
24/24 - 0s - loss: 1.4982 - accuracy: 0.3992 - precision_8: 0.6423 - recall_8: 0.2366 - val_loss: 1.4600 - val_accuracy: 0.4324 - val_precision_8: 0.5862 - val_recall_8: 0.2757
Epoch 37/170
24/24 - 0s - loss: 1.6459 - accuracy:

Epoch 76/170
24/24 - 0s - loss: 1.1730 - accuracy: 0.5390 - precision_8: 0.6809 - recall_8: 0.3642 - val_loss: 1.1925 - val_accuracy: 0.5243 - val_precision_8: 0.7308 - val_recall_8: 0.3081
Epoch 77/170
24/24 - 0s - loss: 1.1360 - accuracy: 0.5672 - precision_8: 0.7309 - recall_8: 0.3468 - val_loss: 1.1780 - val_accuracy: 0.5351 - val_precision_8: 0.6518 - val_recall_8: 0.3946
Epoch 78/170
24/24 - 0s - loss: 1.0536 - accuracy: 0.5981 - precision_8: 0.7194 - recall_8: 0.4341 - val_loss: 1.2695 - val_accuracy: 0.5081 - val_precision_8: 0.6604 - val_recall_8: 0.3784
Epoch 79/170
24/24 - 0s - loss: 1.1317 - accuracy: 0.5645 - precision_8: 0.7154 - recall_8: 0.3683 - val_loss: 1.1273 - val_accuracy: 0.5676 - val_precision_8: 0.7300 - val_recall_8: 0.3946
Epoch 80/170
24/24 - 0s - loss: 1.0374 - accuracy: 0.6250 - precision_8: 0.7549 - recall_8: 0.4180 - val_loss: 1.0996 - val_accuracy: 0.5568 - val_precision_8: 0.6754 - val_recall_8: 0.4162
Epoch 81/170
24/24 - 0s - loss: 0.9808 - accuracy:

24/24 - 0s - loss: 0.6433 - accuracy: 0.7460 - precision_8: 0.8283 - recall_8: 0.6613 - val_loss: 0.8328 - val_accuracy: 0.6973 - val_precision_8: 0.7815 - val_recall_8: 0.6378
Epoch 120/170
24/24 - 0s - loss: 0.5750 - accuracy: 0.7688 - precision_8: 0.8331 - recall_8: 0.7110 - val_loss: 0.8441 - val_accuracy: 0.6973 - val_precision_8: 0.7703 - val_recall_8: 0.6162
Epoch 121/170
24/24 - 0s - loss: 0.5818 - accuracy: 0.7675 - precision_8: 0.8292 - recall_8: 0.7110 - val_loss: 0.8789 - val_accuracy: 0.6811 - val_precision_8: 0.7697 - val_recall_8: 0.6324
Epoch 122/170
24/24 - 0s - loss: 0.6149 - accuracy: 0.7742 - precision_8: 0.8145 - recall_8: 0.7083 - val_loss: 1.1155 - val_accuracy: 0.6324 - val_precision_8: 0.7075 - val_recall_8: 0.5622
Epoch 123/170
24/24 - 0s - loss: 0.7202 - accuracy: 0.7339 - precision_8: 0.7821 - recall_8: 0.6801 - val_loss: 0.9367 - val_accuracy: 0.6811 - val_precision_8: 0.7452 - val_recall_8: 0.6324
Epoch 124/170
24/24 - 0s - loss: 0.8837 - accuracy: 0.6949 

Epoch 162/170
24/24 - 0s - loss: 0.5491 - accuracy: 0.8172 - precision_8: 0.8343 - recall_8: 0.7984 - val_loss: 0.9972 - val_accuracy: 0.7135 - val_precision_8: 0.7384 - val_recall_8: 0.6865
Epoch 163/170
24/24 - 0s - loss: 0.5796 - accuracy: 0.7823 - precision_8: 0.8138 - recall_8: 0.7460 - val_loss: 0.7213 - val_accuracy: 0.7405 - val_precision_8: 0.7811 - val_recall_8: 0.7135
Epoch 164/170
24/24 - 0s - loss: 0.4408 - accuracy: 0.8320 - precision_8: 0.8580 - recall_8: 0.8118 - val_loss: 0.8149 - val_accuracy: 0.7351 - val_precision_8: 0.7892 - val_recall_8: 0.7081
Epoch 165/170
24/24 - 0s - loss: 0.4025 - accuracy: 0.8589 - precision_8: 0.8862 - recall_8: 0.8374 - val_loss: 0.6724 - val_accuracy: 0.7730 - val_precision_8: 0.8166 - val_recall_8: 0.7459
Epoch 166/170
24/24 - 0s - loss: 0.3529 - accuracy: 0.8790 - precision_8: 0.8892 - recall_8: 0.8629 - val_loss: 0.6665 - val_accuracy: 0.7784 - val_precision_8: 0.8103 - val_recall_8: 0.7622
Epoch 167/170
24/24 - 0s - loss: 0.3110 - acc

NameError: name 'numpy' is not defined

[0.6836291551589966, 0.7945945858955383, 0.8265895843505859, 0.7729730010032654]
78.47% (+/- 3.16%)
